# LLM Inference Optimization

**References**
- *Mastering LLM Techniques: Inference Optimization: [Blog](https://developer.nvidia.com/blog/mastering-llm-techniques-inference-optimization/)*



## Introduction

In the decode phase, the LLM generates output tokens one at a time, until a stopping criteria is met. Each sequential output token needs to know all the previous iterations' output states (keys and values). The speed at which the data (weights, keys, values, activations) is transferred to the GPU from memory dominates the latency, not how fast the computation actually happens. In other words, this is a memory-bound operation.

Many of the inference challenges and corresponding solutions in this notebook concern the optimization of this decode phase: efficient attention modules, managing the keys and values effectively, and others.

When comparing inference throughput, even if two LLMs have similar tokens per second output, they may not be equivalent if they use different tokenizers. This is because corresponding tokens may represent a different number of characters. 


## Model Serving

Model execution is frequently memory-bandwidth bound, in particular, bandwidth-bound in the weights.

The simplest way to improve GPU utilization, and effectively throughput, is via batching. When you get multiple requests, you can batch all the requests together and this helps leverage more of the compute available. But there is a limit to how many requests can be in a single batch, after which they may lead to memory overflow.

Unlike traditional batching, batching for LLMs is complex. Since each request in a batch may generate a different number of completion tokens the execution times may end up different. This then causes the batch to wait till the longest request process is completed, which is bad. However, there are ways to fix this, such as in-flight batching.


### In-flight batching

Continuous or in-flight batching takes advantage of the fact that the overall text generation process for an LLM can be broken down into multiple iterations of execution on the model. With in-flight batching, rather than waiting for the whole batch to finish before moving on to the next set of requests, the server runtime immediately evicts finished sequences from the batch. It then begins executing new requests while other requests are still in flight.

### Speculative inference

Also known as speculative sampling, assisted generation, or blockwise parallel decoding, speculative inference is a different way of parallelizing the execution of LLMs.

In autoregressive language models, every token that is generated relies on all of the tokens that come before it to provide context. This means that in regular execution, it’s impossible to generate multiple tokens from the same sequence in parallel—you have to wait for the nth token to be generated before you can generate n+1.

Speculative inference is where you use a temporary draft model to make multiple future steps predictions which are then verified or rejected in parallel.

![Speculative Inference Example](./images/Speculative-Inference-Example.png)

The basic idea of speculative sampling is to use some “cheaper” process to generate a draft continuation that is several tokens long. Then, execute the main “verification” model at multiple steps in parallel, using the cheap draft as “speculative” context for the execution steps where it is needed. If the verification model generates the same tokens as the draft, then you know to accept those tokens for the output. Otherwise, you can throw out everything after the first non-matching token, and repeat the process with a new draft.

There are many different options for how to generate draft tokens, and each comes with different tradeoffs. You can train multiple models, or fine-tune multiple heads on a single pretrained model, that predict tokens that are multiple steps in the future. Or, you can use a small model as the draft model, and a larger, more capable model as the verifier.


## Key-Value Caching

During the decode phase, to generate a single token, you need the key and value tensors of all the previous tokens (including the input tokens’ KV tensors computed at training, and any new KV tensors computed until the current time step). To avoid recomputing all these tensors for all tokens at each time step, it’s possible to cache them in GPU memory.

Every iteration, when new elements are computed, they are simply added to the running cache to be used in the next iteration. In some implementations, there is one KV cache for each layer of the model.

The two main contributors to the GPU LLM memory requirement are model weights and the KV cache.
- Model weights: for example a model with 7B parameters, loaded in 16-bit precision (FP16 or BF16) would roughtly take: $7B \times \text{size of (FP16)} \approx 14GB$ in memory.
- KV Caching: from caching self-attention tensors

With batching, the KV cache of each of the requests in the batch must still be allocated separately, and can have a large memory footprint. To calculate the size of the KV cache:
$$\text{Size of KV Cache per token in bytes} = 2 \times (\text{num layers}) \times (\text{num heads} \times \text{dim head}) \times \text{precision in bytes}$$

Where:
- 2 accounts for the K and V matrices
- usually the value of (num_heads * dim_head) is the same as the hidden_size (or dimension of the model, d_model) of the transformer

This memory size is required for each token in the input sequence, across the batch of inputs. Assuming half-precision, the total size of KV cache:
$$\text{Total size of KV Cache in bytes} = (\text{batch size}) \times (\text{sequence length}) \times 2 \times (\text{num layers}) \times (\text{hidden size}) \times \text{size of (FP16)}$$

For example, for Llama2 7B model in a 16-bit precision and a batch size of 1, the size of KV cache: $(1) \times (4096) \times 2 \times (32) \times (4096) * 2\text{bytes} \approx 2GB$

If you notice, the KV cache memory requirement grows linearly with batch size and sequence length, thereby limiting the throughput the can be served and posing challenges for long-context inputs.


### PagedAttention

KV Cache are usually over-provisioned to account for the largest possible input because the size of inputs is unpredictable. For example if a model supports a max sequence length of 2048, then regardless of the size of the input and the generated output in a request, we reserve a size of 2048. This space may be contiguously allocated, and often, much of it remains unused, leading to memory waste or fragmentation. This reserved space is tied up for the lifetime of the request.

Inspired by paging in operating systems, the PagedAttention algorithm enables storing continuous keys and values in noncontiguous space in memory. It partitions the KV cache of each request into blocks representing a fixed number of tokens, which can be stored non-contiguously. 

These blocks are fetched as required during attention computation using a block table that keeps account. As new tokens are generated, new block allocations are made. The size of these blocks is fixed, eliminating inefficiencies arising from challenges like different requests requiring different allocations. This significantly limits memory wastage, enabling larger batch sizes (and, consequently, throughput).


## Model Parallelism

### Pipeline Parallelism

Pipeline parallelism involves sharding the model (vertically) into chunks, where each chunk comprises a subset of layers that is executed on a separate device.

![Pipeline Parallelism Fig 1](./images/pipeline-parallelism-fig1.png)

In the figure above, F represents a forward pass in different devices and B represents the backward pass. This effectively reduces the memory required to store a large model by splitting the model layers into seperate devices. The main limitation of this method is that, due to the sequential nature of the processing, some devices or layers may remain idle while waiting for the output (activations, gradients) of previous layers.

The figure below shows the inefficiency with respect to time:

![Pipeline Parallelism Fig 2](./images/pipeline-parallelism-fig2.png)

This can be partially fixed by using micro-batching.

![Pipeline Parallelism Fig 3](./images/pipeline-parallelism-fig3.png)


### Tensor Parallelism

Tensor parallelism involves sharding (horizontally) individual layers of the model into smaller, independent blocks of computation that can be executed on different devices. Attention blocks and multi-layer perceptron (MLP) layers are major components of transformers that can take advantage of tensor parallelism. In multi-head attention blocks, each head or group of heads can be assigned to a different device so they can be computed independently and in parallel.


### Sequence Parallelism

Tensor parallelism has limitations, as it requires layers to be divided into independent, manageable blocks. It’s not applicable to operations like LayerNorm and Dropout, which are instead replicated across the tensor-parallel group. While LayerNorm and Dropout are computationally inexpensive, they do require a considerable amount of memory to store (redundant) activations.

As shown in [Reducing Activation Recomputation in Large Transformer Models](https://arxiv.org/abs/2205.05198), these operations are independent across the input sequence, and these ops can be partitioned along that “sequence-dimension,” making them more memory efficient. This is called sequence parallelism.

Techniques for model parallelism are not exclusive and can be used in conjunction.

## Attention Mechanism Optimization

### Flash Attention

## Model Optimization

### Quantization

There are many different quantization techniques for LLMs involving reduced precision on either the activations, the weights, or both. It’s much more straightforward to quantize the weights because they are fixed after training. However, this can leave some performance on the table because the activations remain at higher precisions. GPUs don’t have dedicated hardware for multiplying INT8 and FP16 numbers, so the weights must be converted back into a higher precision for the actual operations. 

It’s also possible to quantize the activations, the inputs of transformer blocks and network layers, but this comes with its own challenges. Activation vectors often contain outliers, effectively increasing their dynamic range and making it more challenging to represent these values at a lower precision than with the weights. 

One option is to find out where those outliers are likely to show up by passing a representative dataset through the model, and choosing to represent certain activations at a higher precision than others (LLM.int8()). Another option is to borrow the dynamic range of the weights, which are easy to quantize, and reuse that range in the activations.


### Sparsity

Similar to quantization, it’s been shown that many deep learning models are robust to pruning, or replacing certain values that are close to 0 with 0 itself. Sparse matrices are matrices where many of the elements are 0. These can be expressed in a condensed form that takes up less space than a full, dense matrix.

![Sparse Matrix Representation](./images/sparse-matrix-representation.png)

GPUs in particular have hardware acceleration for a certain kind of structured sparsity, where two out of every four values are represented by zeros. Sparse representations can also be combined with quantization to achieve even greater speedups in execution.


### Distillation

Distillation involves training a smaller model (called a student) to mimic the behavior of a larger model (a teacher). The figure shows a general framework for knowledge distillation. The logits of the teacher are soft targets that the student optimizes for using a distillation loss. Other distillation methods may use other measures of loss to “distill” knowledge from the teacher.

![A general framework for knowledge distillation](./images/knowledge-distillation-general-framework.png)

An alternative approach to distillation is to use data synthesized by the teacher for supervised training of a student LLM, which is especially useful when human annotations are scarce or not available. [Distilling Step by Step](https://arxiv.org/abs/2305.02301) goes one step further by extracting rationales from a teacher LLM in addition to the labels that serve as ground truth. These rationales serve as intermediate reasoning steps to train smaller student LLMs in a data-efficient way.